## **Applio**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio)

<br>

### **Credits**
- Encryption method: [Hina](https://github.com/hinabl)
- Extra section: [Poopmaster](https://github.com/poiqazwsx)
- Uv code: [Shirou](https://github.com/ShiromiyaG)
- Main development: [Applio Team](https://github.com/IAHispano)



In [ ]:
# @title **Install Applio**
import codecs
from IPython.display import clear_output

rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

org_name = rot_47("Vkkgdj")
new_name = rot_47("kmjbmvh_hg")
uioawhd = rot_47(codecs.decode("pbbxa://oqbpcj.kwu/QIPqaxivw/Ixxtqw.oqb", "rot_13"))
uyadwa = codecs.decode("ncc.cl", "rot_13")
A = "/content/" + rot_47("Kikpm.ovm.bu")
!pip install uv pyngrok
!git clone --depth 1 $uioawhd $new_name --branch 3.2.8-bugfix --single-branch
%cd $new_name/
!uv venv .venv --python 3.10.12
clear_output()
print("Installing requirements...")
!uv pip install -r requirements.txt -q
!uv pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --upgrade --index-url https://download.pytorch.org/whl/cu121 -q
!uv pip install numpy==1.23.5 -q
clear_output()
print("Finished installing requirements! ")

In [ ]:
# @title **Start Applio**
# @markdown  ### Choose a sharing method:

import codecs
import threading
import urllib.request
import time
import ipywidgets as widgets
from IPython.display import display
import os
from pyngrok import ngrok
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]

ngrok_token = "empty" # @param {type:"string"}

def start_gradio():
    !uv run $uyadwa --listen --share

def start_localtunnel():
    !npm install -g localtunnel &>/dev/null
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 6969 >> url.txt 2>&1 &')
    time.sleep(2)
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")

    print(f"Share Link: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m\n")

    password_endpoint_widget = widgets.Text(
    value=endpoint_ip,
    description='Password IP:',
    disabled=True
    )
    display(password_endpoint_widget)
    !uv run $uyadwa --listen


def start_ngrok():
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(6969)
        print(f"Ngrok URL: \033[0m\033[93m{tunnel.public_url}\033[0m", end="\033[0m\n")
        !uv run $uyadwa --listen
    except Exception as e:
        print(f"Error starting ngrok: {e}")

def start_applio():
    if method == 'gradio':
        start_gradio()
    elif method == 'localtunnel':
        start_localtunnel()
    elif method == 'ngrok':
        start_ngrok()

%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs --bind_all

if "autobackups" not in globals():
    autobackups = False

if autobackups:
    thread = threading.Thread(target=backup_files)
    thread.start()

thread_applio = threading.Thread(target=start_applio)
thread_applio.start()


while True:
    time.sleep(5)

### **Extra**
Enjoy extra options that can make it easier for you to use Applio


In [ ]:
# @title Mount Drive
# @markdown Mount the files from Google Drive to the Colab.
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Auto Backup
# @markdown When running it, it will be activated or deactivated previously to start up together with Applio.
import os
import shutil
import time

LOGS_FOLDER = "/content/program_ml/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/ApplioBackup"

if "autobackups" not in globals():
    autobackups = False

cooldown = 15  # @param {type:"slider", min:0, max:100, step:0}
def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" and "mute" directories
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1


            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]


            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)


        except Exception as error:
            print(f"An error occurred during backup: {error}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

In [ ]:
# @title Load a Backup
from google.colab import drive
import os

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "My-Project"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/ApplioBackup/" + modelname
destination_path = "/content/program_ml/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/ApplioBackup/", backup_timestamps_file)
    time__ = os.path.join("/content/program_ml/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
# @title Download all custom pretrains
import os
import urllib.request

%mkdir /content/program_ml/rvc/models/pretraineds/pretraineds_custom
pretrained_urls = [
    # Ov2 Super
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kG.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kD.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kG.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kD.pth",

    # TITAN
    "https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/40k/pretrained/G-f040k-TITAN-Medium.pth",
    "https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/40k/pretrained/D-f040k-TITAN-Medium.pth",
    "https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/32k/pretrained/G-f032k-TITAN-Medium.pth",
    "https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/32k/pretrained/D-f032k-TITAN-Medium.pth",

    # Snowie V3
    "https://huggingface.co/MUSTAR/SnowieV3.1-32k/resolve/main/D_SnowieV3.1_32k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-32k/resolve/main/G_SnowieV3.1_32k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-40k/resolve/main/G_SnowieV3.1_40k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-40k/resolve/main/D_SnowieV3.1_40k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-48k/resolve/main/G_SnowieV3.1_48k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-48k/resolve/main/D_SnowieV3.1_48k.pth",

    # RIN E3
    "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_G.pth",
    "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_D.pth",

    # KLM
    "https://huggingface.co/SeoulStreamingStation/KLM4.1/resolve/main/D_KLM41_32k.pth",
    "https://huggingface.co/SeoulStreamingStation/KLM4.1/resolve/main/G_KLM41_32k.pth",
    "https://huggingface.co/SeoulStreamingStation/KLM4.1/resolve/main/D_KLM41_48k.pth",
    "https://huggingface.co/SeoulStreamingStation/KLM4.1/resolve/main/G_KLM41_48k.pth",

    # SnowieV3 X RIN_E3
    "https://huggingface.co/MUSTAR/SnowieV3.1-X-RinE3-40K/resolve/main/D_Snowie-X-Rin_40k.pth",
    "https://huggingface.co/MUSTAR/SnowieV3.1-X-RinE3-40K/resolve/main/G_Snowie-X-Rin_40k.pth",
]
output_directory = "/content/program_ml/rvc/models/pretraineds/pretraineds_custom"
for url in pretrained_urls:
    filename = os.path.join(output_directory, os.path.basename(url))
    urllib.request.urlretrieve(url, filename)